In [1]:
# Импорт всякого

import os
import pandas as pd
import numpy as np

In [2]:
# Загрузка

PATH_2_DATA = "Машины/downloaded/CarPrice_Assignment.csv"

cars = pd.read_csv(PATH_2_DATA)

In [3]:
# обзор данных в консоли

pd.options.display.max_columns = cars.shape[1]
print(cars.head())
cars.info() 
cars.columns

   car_ID  symboling                   CarName fueltype aspiration doornumber  \
0       1          3        alfa-romero giulia      gas        std        two   
1       2          3       alfa-romero stelvio      gas        std        two   
2       3          1  alfa-romero Quadrifoglio      gas        std        two   
3       4          2               audi 100 ls      gas        std       four   
4       5          2                audi 100ls      gas        std       four   

       carbody drivewheel enginelocation  wheelbase  carlength  carwidth  \
0  convertible        rwd          front       88.6      168.8      64.1   
1  convertible        rwd          front       88.6      168.8      64.1   
2    hatchback        rwd          front       94.5      171.2      65.5   
3        sedan        fwd          front       99.8      176.6      66.2   
4        sedan        4wd          front       99.4      176.6      66.4   

   carheight  curbweight enginetype cylindernumber  engi

Index(['car_ID', 'symboling', 'CarName', 'fueltype', 'aspiration',
       'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'wheelbase',
       'carlength', 'carwidth', 'carheight', 'curbweight', 'enginetype',
       'cylindernumber', 'enginesize', 'fuelsystem', 'boreratio', 'stroke',
       'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg',
       'price'],
      dtype='object')

In [4]:
# car_ID - уникальное поле, убираем

if "car_ID" in cars.columns:

    print(cars["car_ID"].describe())
    print(cars["car_ID"].value_counts())
    print(cars["car_ID"].dtype)

    print(len(set(cars["car_ID"])) == len(cars["car_ID"])) # проверка на уникальность

    del cars["car_ID"]

count    205.000000
mean     103.000000
std       59.322565
min        1.000000
25%       52.000000
50%      103.000000
75%      154.000000
max      205.000000
Name: car_ID, dtype: float64
1      1
142    1
132    1
133    1
134    1
      ..
72     1
73     1
74     1
75     1
205    1
Name: car_ID, Length: 205, dtype: int64
int64
True


In [5]:
# имеет смысл оставить компанию производителя от названия в виде one-hot-encoding'а
# интуитивно понятно, что полное название тоже влияет на цену,
# но чтобы перевести его в цифру, нужно использовать специальные инструменты, а-ля word2vec

print(cars["CarName"].describe())
print(cars["CarName"].value_counts())
print(cars["CarName"].dtype)

CarName = cars.loc[:, "CarName"]

for i in range(len(CarName)):
    CarName.iloc[i] = CarName.iloc[i].split()[0]

errors = {"maxda" : "mazda",
          "porcshce" : "porsche",
          "toyouta" : "toyota",
          "vokswagen" : "volkswagen",
          "vw" : "volkswagen" }

for i in range(len(CarName)):
    name = str(CarName.iloc[i])

    if name in errors:
        CarName.iloc[i] = errors[name]

def to_one_hot(df, column):
    if column not in df.columns:
        return
    
    labels = list(set(df[column]))

    df[labels] = np.zeros((len(df[column]), len(labels)), dtype=np.int32)

    for i in range(len(df[column])):
        df[df[column].iloc[i]].iloc[i] = 1

    del df[column]

to_one_hot(cars, "CarName")

count               205
unique              147
top       toyota corona
freq                  6
Name: CarName, dtype: object
toyota corona                   6
toyota corolla                  6
peugeot 504                     6
subaru dl                       4
mazda 626                       3
                               ..
volkswagen 1131 deluxe sedan    1
buick electra 225 custom        1
porcshce panamera               1
Nissan versa                    1
mitsubishi montero              1
Name: CarName, Length: 147, dtype: int64
object


E:\Anaconda\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [6]:
# fueltype - категориальный признак -> 0, 1

print(cars["fueltype"].describe())
print(set(cars["fueltype"]))

cars["fueltype"] = cars["fueltype"].map({'gas' : 1, 'diesel': 0})
cars = cars.rename(columns={ "fueltype" : "gas"})

count     205
unique      2
top       gas
freq      185
Name: fueltype, dtype: object
{'diesel', 'gas'}


In [7]:
# aspiration -> 0, 1

print(cars["aspiration"].describe())
print(set(cars["aspiration"]))

cars["aspiration"] = cars["aspiration"].map({'turbo' : 1, 'std': 0})
cars = cars.rename(columns={ "aspiration" : "turbo"})

count     205
unique      2
top       std
freq      168
Name: aspiration, dtype: object
{'turbo', 'std'}


In [8]:
# doornumber -> 4, 2

print(cars["doornumber"].describe())
print(set(cars["doornumber"]))

cars["doornumber"] = cars["doornumber"].map({'four' : 4, 'two': 2})

count      205
unique       2
top       four
freq       115
Name: doornumber, dtype: object
{'four', 'two'}


In [9]:
# carbody

print(cars["carbody"].describe())
print(set(cars["carbody"]))

to_one_hot(cars, "carbody")

count       205
unique        5
top       sedan
freq         96
Name: carbody, dtype: object
{'wagon', 'convertible', 'sedan', 'hardtop', 'hatchback'}


E:\Anaconda\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
# drivewheel

print(cars["drivewheel"].describe())
print(set(cars["drivewheel"]))

to_one_hot(cars, "drivewheel")

count     205
unique      3
top       fwd
freq      120
Name: drivewheel, dtype: object
{'fwd', 'rwd', '4wd'}


E:\Anaconda\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [11]:
# enginelocation

print(cars["enginelocation"].describe())
print(set(cars["enginelocation"]))

cars["enginelocation"] = cars["enginelocation"].map({'rear' : 1, 'front': 0})
cars["enginelocation"]

count       205
unique        2
top       front
freq        202
Name: enginelocation, dtype: object
{'front', 'rear'}


0      0
1      0
2      0
3      0
4      0
      ..
200    0
201    0
202    0
203    0
204    0
Name: enginelocation, Length: 205, dtype: int64

In [12]:
# enginetype - one-hot

print(cars["enginetype"].describe())
print(set(cars["enginetype"]))

to_one_hot(cars, "enginetype")

count     205
unique      7
top       ohc
freq      148
Name: enginetype, dtype: object
{'ohc', 'ohcf', 'rotor', 'l', 'dohc', 'ohcv', 'dohcv'}


E:\Anaconda\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [13]:
# cylindernumber

print(cars["cylindernumber"].describe())
print(set(cars["cylindernumber"]))

cars["cylindernumber"] = cars["cylindernumber"].map({'four' : 4, 'six': 6, 'twelve': 12, 'eight': 8, 'five': 5, 'three': 3, 'two': 2 })

count      205
unique       7
top       four
freq       159
Name: cylindernumber, dtype: object
{'twelve', 'three', 'four', 'six', 'eight', 'five', 'two'}


In [14]:
# fuelsystem

print(cars["fuelsystem"].describe())
print(set(cars["fuelsystem"]))

to_one_hot(cars, "fuelsystem")

count      205
unique       8
top       mpfi
freq        94
Name: fuelsystem, dtype: object
{'4bbl', 'spfi', 'spdi', '2bbl', 'mpfi', '1bbl', 'mfi', 'idi'}


E:\Anaconda\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [15]:
# Проверяем, что не осталось категориальных признаков

types = [None] * len(cars.columns)
for i, column in enumerate(cars.columns):
    types[i] = cars[column].dtype

set(types)


{dtype('int64'), dtype('float64')}

In [16]:
# Разбиваем данные на тренировочную и тестовую выборки по атрибуту price

def split_data_ind(df, column, test_size = 0.1, n_splits = 10):
    data = np.array(df[column])
    pockets = [[]] * n_splits # карманы

    Min = np.min(data)
    Max = np.max(data)

    for i in range(len(data)): # отправляем индексы по карманам
        pockets[min(int(np.floor((data[i] - Min) / (Max - Min) * n_splits)), n_splits - 1)].append(i)
    
    train_ind = []
    test_ind = []

    for i in range(n_splits):
        rand = np.random.uniform(0, 1, len(pockets[i]))

        for j in range(len(pockets[i])):
            if rand[j] <= test_size:
                test_ind.append(pockets[i][j])
            else:
                train_ind.append(pockets[i][j])

    return train_ind, test_ind

train_ind, test_ind = split_data_ind(cars, "price")

train_set = cars.iloc[np.random.permutation(train_ind), :]
test_set = cars.iloc[np.random.permutation(test_ind), :]

train_set.to_csv("Машины/prepared/train.csv", index=False)
test_set.to_csv("Машины/prepared/test.csv", index=False)
